In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder \
.appName('Large-Scale Order Processing System') \
.getOrCreate()

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Section 1 — Data Ingestion & Schema

1.  Read the CSV le using PySpark ensuring the job does not fail due to bad data.


In [3]:
df_raw = spark.read\
.option("header", True) \
.option("badRecordsPath", "/tmp/bad_records") \
.csv("/content/orders_raw.csv")

2. Explain why reading all columns as StringType is preferred initially.


In [4]:
#When working with raw data, especially from sources like CSVs, it's often best practice to initially load all columns as StringType. This approach offers several significant advantages for data quality and robust processing.

3. Print schema and total record count.

In [5]:
df_raw.printSchema()
df_raw.count()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)



300000

#Section 2 — Data Cleaning & Validation

4. Clean leading/trailing spaces from string columns.


In [6]:
from pyspark.sql.functions import col, trim
df = df_raw.select([trim(col(c)).alias(c) for c in df_raw.columns])
df.show()

+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|     city|   category|    product| amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|ORD00000000|    C000000|hyderabad|    grocery|        Oil|invalid|01/01/2024|Cancelled|
|ORD00000001|    C000001|     Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|
|ORD00000002|    C000002|     Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|
|ORD00000003|    C000003|Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed|
|ORD00000004|    C000004|     Pune|       Home|AirPurifier|  33659|2024-01-05|Completed|
|ORD00000005|    C000005|    Delhi|    Fashion|      Jeans|   8521|2024-01-06|Completed|
|ORD00000006|    C000006|    Delhi|    Grocery|      Sugar|  42383|2024-01-07|Completed|
|ORD00000007|    C000007|     Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|
|ORD00000008|    C000

5. Standardize city , category , and product values.


In [7]:
from pyspark.sql.functions import lower, initcap
df = df.withColumn("city", initcap(lower(col("city"))))\
.withColumn("category", lower(col("category")))\
.withColumn("product", lower(col("product")))

df.show()

+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|     city|   category|    product| amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|ORD00000000|    C000000|Hyderabad|    grocery|        oil|invalid|01/01/2024|Cancelled|
|ORD00000001|    C000001|     Pune|    grocery|      sugar|  35430|2024-01-02|Completed|
|ORD00000002|    C000002|     Pune|electronics|     mobile|  65358|2024-01-03|Completed|
|ORD00000003|    C000003|Bangalore|electronics|     laptop|   5558|2024-01-04|Completed|
|ORD00000004|    C000004|     Pune|       home|airpurifier|  33659|2024-01-05|Completed|
|ORD00000005|    C000005|    Delhi|    fashion|      jeans|   8521|2024-01-06|Completed|
|ORD00000006|    C000006|    Delhi|    grocery|      sugar|  42383|2024-01-07|Completed|
|ORD00000007|    C000007|     Pune|    grocery|       rice|  45362|2024-01-08|Completed|
|ORD00000008|    C000

6. Convert amount to integer safely, handling invalid values.


In [8]:
from pyspark.sql.functions import col, regexp_replace, nullif, lit
from pyspark.sql.types import IntegerType

# Remove all non-digit characters from the 'amount' column
cleaned_amount_col = regexp_replace(col("amount"), "[^0-9]", "")

# Convert empty strings (e.g., from 'invalid' or non-numeric entries) to NULL
nullable_amount_col = nullif(cleaned_amount_col, lit(""))

# Cast the cleaned and null-handled column to IntegerType
df = df.withColumn("amount", nullable_amount_col.cast(IntegerType()))

df.show()

+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|   order_id|customer_id|     city|   category|    product|amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|ORD00000000|    C000000|Hyderabad|    grocery|        oil|  NULL|01/01/2024|Cancelled|
|ORD00000001|    C000001|     Pune|    grocery|      sugar| 35430|2024-01-02|Completed|
|ORD00000002|    C000002|     Pune|electronics|     mobile| 65358|2024-01-03|Completed|
|ORD00000003|    C000003|Bangalore|electronics|     laptop|  5558|2024-01-04|Completed|
|ORD00000004|    C000004|     Pune|       home|airpurifier| 33659|2024-01-05|Completed|
|ORD00000005|    C000005|    Delhi|    fashion|      jeans|  8521|2024-01-06|Completed|
|ORD00000006|    C000006|    Delhi|    grocery|      sugar| 42383|2024-01-07|Completed|
|ORD00000007|    C000007|     Pune|    grocery|       rice| 45362|2024-01-08|Completed|
|ORD00000008|    C000008|Bangalo

7. Parse order_date supporting multiple date formats.


In [27]:
from pyspark.sql import functions as F

df = df.withColumn(
    "order_date_clean",
    F.coalesce(
        F.expr("try_to_timestamp(order_date, 'yyyy-MM-dd')").cast("date"),
        F.expr("try_to_timestamp(order_date, 'yyyy/MM/dd')").cast("date"),
        F.expr("try_to_timestamp(order_date, 'dd/MM/yyyy')").cast("date"),
        F.expr("try_to_timestamp(order_date, 'dd-MM-yyyy')").cast("date")
    )
).drop("order_date", "order_date_parsed").withColumnRenamed("order_date_clean", "order_date")

df.select("order_date").show(20, truncate=False)
df.printSchema()

+----------+
|order_date|
+----------+
|2024-01-01|
|2024-01-02|
|2024-01-03|
|2024-01-04|
|2024-01-05|
|2024-01-06|
|2024-01-07|
|2024-01-08|
|2024-01-09|
|2024-01-10|
|2024-01-11|
|2024-01-12|
|2024-01-13|
|2024-01-14|
|2024-01-15|
|2024-01-16|
|2024-01-17|
|2024-01-18|
|2024-01-19|
|2024-01-20|
+----------+
only showing top 20 rows
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date: date (nullable = true)



8. Identify and handle invalid or null records.

In [29]:
print(f"Total records after cleaning and validation: {df_valid.count()}")
invalid_records_count = df.count() - df_valid.count()
print(f"Number of invalid/null records dropped: {invalid_records_count}")

Total records after cleaning and validation: 272458
Number of invalid/null records dropped: 27542


#Section 3 — Business Rules


9. Remove duplicate records based on order_id .


In [28]:
df_valid = df.filter(
    F.col("order_id").isNotNull() &
    F.col("order_date").isNotNull() &
    F.col("amount").isNotNull()
)
df_valid.show()

+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|   order_id|customer_id|     city|   category|    product|amount|   status|order_date|
+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|ORD00000001|    C000001|     Pune|    grocery|      sugar| 35430|Completed|2024-01-02|
|ORD00000002|    C000002|     Pune|electronics|     mobile| 65358|Completed|2024-01-03|
|ORD00000003|    C000003|Bangalore|electronics|     laptop|  5558|Completed|2024-01-04|
|ORD00000004|    C000004|     Pune|       home|airpurifier| 33659|Completed|2024-01-05|
|ORD00000005|    C000005|    Delhi|    fashion|      jeans|  8521|Completed|2024-01-06|
|ORD00000006|    C000006|    Delhi|    grocery|      sugar| 42383|Completed|2024-01-07|
|ORD00000007|    C000007|     Pune|    grocery|       rice| 45362|Completed|2024-01-08|
|ORD00000008|    C000008|Bangalore|    fashion|      jeans| 10563|Completed|2024-01-09|
|ORD00000009|    C000009|  Kolka

In [30]:
df_dedup = df_valid.dropDuplicates(["order_id"])
df_dedup.show(20, truncate=False)
df_dedup.count()

+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|order_id   |customer_id|city     |category   |product    |amount|status   |order_date|
+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|ORD00000001|C000001    |Pune     |grocery    |sugar      |35430 |Completed|2024-01-02|
|ORD00000007|C000007    |Pune     |grocery    |rice       |45362 |Completed|2024-01-08|
|ORD00000008|C000008    |Bangalore|fashion    |jeans      |10563 |Completed|2024-01-09|
|ORD00000010|C000010    |Bangalore|grocery    |sugar      |66576 |Completed|2024-01-11|
|ORD00000011|C000011    |Kolkata  |electronics|tablet     |50318 |Completed|2024-01-12|
|ORD00000012|C000012    |Bangalore|grocery    |sugar      |84768 |Completed|2024-01-13|
|ORD00000014|C000014    |Mumbai   |electronics|tablet     |79469 |Completed|2024-01-15|
|ORD00000015|C000015    |Pune     |electronics|mobile     |81018 |Completed|2024-01-16|
|ORD00000017|C000017    |Bangalo

272458

10. Filter only records with status = Completed .


In [33]:
df_completed = df_dedup.filter(col("status") == "Completed")
df_completed.show()

+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|   order_id|customer_id|     city|   category|    product|amount|   status|order_date|
+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|ORD00000001|    C000001|     Pune|    grocery|      sugar| 35430|Completed|2024-01-02|
|ORD00000007|    C000007|     Pune|    grocery|       rice| 45362|Completed|2024-01-08|
|ORD00000008|    C000008|Bangalore|    fashion|      jeans| 10563|Completed|2024-01-09|
|ORD00000010|    C000010|Bangalore|    grocery|      sugar| 66576|Completed|2024-01-11|
|ORD00000011|    C000011|  Kolkata|electronics|     tablet| 50318|Completed|2024-01-12|
|ORD00000012|    C000012|Bangalore|    grocery|      sugar| 84768|Completed|2024-01-13|
|ORD00000014|    C000014|   Mumbai|electronics|     tablet| 79469|Completed|2024-01-15|
|ORD00000015|    C000015|     Pune|electronics|     mobile| 81018|Completed|2024-01-16|
|ORD00000017|    C000017|Bangalo

11. Validate record counts before and after ltering.

In [32]:
print(f"Total records after cleaning and validation: {df_valid.count()}")

Total records after cleaning and validation: 272458


#Section 4 — Performance & Optimization


12. Identify operations that cause shu es.


In [ ]:
#groupBY, dropDuplicates, join, orderBy, window functions

13. Use explain(True) to analyze the execution plan.


In [34]:
df_dedup.explain(True)

== Parsed Logical Plan ==
Deduplicate [order_id#37]
+- Filter ((isnotnull(order_id#37) AND isnotnull(order_date#1467)) AND isnotnull(amount#115))
   +- Project [order_id#37, customer_id#38, city#79, category#80, product#81, amount#115, status#44, order_date_clean#1466 AS order_date#1467]
      +- Project [order_id#37, customer_id#38, city#79, category#80, product#81, amount#115, status#44, order_date_clean#1466]
         +- Project [order_id#37, customer_id#38, city#79, category#80, product#81, amount#115, status#44, order_date_parsed#150, order_date#1305, coalesce(cast(try_to_timestamp(order_date#1305, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#1305, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#1305, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false) as date), cast(try_to_timestamp(order_date#1305, Some(dd-MM-yyyy), TimestampType, Some(Etc/UTC), false) as date)) AS order_dat

14. Apply repartitioning to optimize aggregations.


In [35]:
df_completed.repartition(col("city")).show()

+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|   order_id|customer_id|     city|   category|    product|amount|   status|order_date|
+-----------+-----------+---------+-----------+-----------+------+---------+----------+
|ORD00000008|    C000008|Bangalore|    fashion|      jeans| 10563|Completed|2024-01-09|
|ORD00000010|    C000010|Bangalore|    grocery|      sugar| 66576|Completed|2024-01-11|
|ORD00000012|    C000012|Bangalore|    grocery|      sugar| 84768|Completed|2024-01-13|
|ORD00000017|    C000017|Bangalore|    grocery|        oil| 69582|Completed|2024-01-18|
|ORD00000024|    C000024|Bangalore|       home|      mixer| 18082|Completed|2024-01-25|
|ORD00000025|    C000025|Bangalore|       home|airpurifier| 58248|Completed|2024-01-26|
|ORD00000124|    C000124|Bangalore|    grocery|      sugar| 54296|Completed|2024-01-05|
|ORD00000159|    C000159|Bangalore|electronics|     tablet| 89397|Completed|2024-02-09|
|ORD00000161|    C000161|Bangalo

15. Justify where caching should be applied and why.

In [36]:
df_completed.cache()

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: int, status: string, order_date: date]

#Section 5 — Analytics





16. Calculate total revenue per city.


In [37]:
from pyspark.sql.functions import sum
df_completed.groupBy("city").agg(sum("amount").alias("total_revenue")).show()

+---------+-------------+
|     city|total_revenue|
+---------+-------------+
|Bangalore|   1614502422|
|  Chennai|   1614928816|
|   Mumbai|   1612372385|
|  Kolkata|   1610016642|
|     Pune|   1631286681|
|    Delhi|   1623209640|
|Hyderabad|   1629316412|
+---------+-------------+



17. Calculate total revenue per category.


In [38]:
revenue_per_category = df_completed.groupBy("category").agg(sum("amount").alias("total_revenue"))
revenue_per_category.show()

+-----------+-------------+
|   category|total_revenue|
+-----------+-------------+
|    grocery|   2841412668|
|electronics|   2842335554|
|       home|   2842762769|
|    fashion|   2809122007|
+-----------+-------------+



18. Identify top 5 products by revenue.


In [39]:
revenue_product = df_completed.groupBy("product").agg(sum("amount").alias("total_revenue"))\
.orderBy(col("total_revenue").desc())
revenue_product.show(5)

+-------+-------------+
|product|total_revenue|
+-------+-------------+
|    oil|    955294283|
| laptop|    955061707|
| tablet|    951653603|
| vacuum|    950475394|
|  mixer|    948801483|
+-------+-------------+
only showing top 5 rows


19. Calculate average order value per city

In [40]:
from pyspark.sql.functions import avg
average_order_value_per_city = df_completed.groupBy("city").agg(avg("amount").alias("average_order_value"))
average_order_value_per_city.show()

+---------+-------------------+
|     city|average_order_value|
+---------+-------------------+
|Bangalore|  44089.20019661924|
|  Chennai|  43606.65377760976|
|   Mumbai| 43722.981397619114|
|  Kolkata|  43719.56340628904|
|     Pune|  43917.90547598535|
|    Delhi| 43792.414611773595|
|Hyderabad|  43718.91198883761|
+---------+-------------------+



#Section 6 — Window Functions


20. Rank cities by total revenue.


In [41]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank
window_city = Window.orderBy(col("total_revenue").desc())
ranked_cities = revenue_per_category.withColumn("rank", rank().over(window_city))
ranked_cities.show()

+-----------+-------------+----+
|   category|total_revenue|rank|
+-----------+-------------+----+
|       home|   2842762769|   1|
|electronics|   2842335554|   2|
|    grocery|   2841412668|   3|
|    fashion|   2809122007|   4|
+-----------+-------------+----+



21. Rank products within each category by revenue.


In [43]:
window_cat = Window.partitionBy("category").orderBy(col("total_revenue").desc())
ranked_products = df_completed.groupBy("category", "product").agg(sum("amount").alias("total_revenue"))\
.withColumn("rank", rank().over(window_cat))
ranked_products.show()

+-----------+-----------+-------------+----+
|   category|    product|total_revenue|rank|
+-----------+-----------+-------------+----+
|electronics|     laptop|    955061707|   1|
|electronics|     tablet|    951653603|   2|
|electronics|     mobile|    935620244|   3|
|    fashion|      jeans|    942574809|   1|
|    fashion|      shoes|    937880299|   2|
|    fashion|     tshirt|    928666899|   3|
|    grocery|        oil|    955294283|   1|
|    grocery|       rice|    945818225|   2|
|    grocery|      sugar|    940300160|   3|
|       home|     vacuum|    950475394|   1|
|       home|      mixer|    948801483|   2|
|       home|airpurifier|    943485892|   3|
+-----------+-----------+-------------+----+



22. Identify the top product per category.

In [44]:
top_product_per_category = ranked_products.filter(col("rank") == 1).drop("rank")
top_product_per_category.show()

+-----------+-------+-------------+
|   category|product|total_revenue|
+-----------+-------+-------------+
|electronics| laptop|    955061707|
|    fashion|  jeans|    942574809|
|    grocery|    oil|    955294283|
|       home| vacuum|    950475394|
+-----------+-------+-------------+



#Section 7 — Storage Strategy


23. Write the cleaned dataset to Parquet partitioned by city .


In [45]:
df_completed.write \
.partitionBy("city") \
.mode("overwrite") \
.parquet("/content/orders_cleaned.parquet")

24. Write aggregated analytics to ORC.


In [46]:
revenue_per_category.write\
.mode("overwrite") \
.orc("/content/orders_analytics.orc")

25. Explain why CSV is not suitable for analytics output.


In [47]:
#no schema
#no compression
#no predicate pushdown
#slow scans
#larger storage

#Section 8 — Debugging & Reasoning


26. Explain why the following line causes failure:
df = df.filter(df.amount > 50000).show()


In [51]:
#If 'amount' is still of StringType (before our cleaning step), comparing it to an Integer (50000) forces Spark to cast the column to Integer implicitly.
#If the column contains non-numeric strings (like 'invalid'), the cast fails.
#In ANSI mode (spark.sql.ansi.enabled=true), this throws a NumberFormatException and stops the job.
#Even without ANSI mode, it converts values to NULL, leading to incorrect results.

27. Describe how you would debug a slow Spark job.


In [ ]:
#1. Spark UI: Check 'Stages' to identify which specific stage is lagging.
#2. Skew Analysis: Look at the task duration distribution. If Max Duration >> Median Duration, a single task is stuck on skewed data (e.g., one city has 90% of orders).

28. Identify risks of over-caching DataFrames

In [50]:
#memory pressure
#disk spills
#slower GC
#eviction of useful cached data